# Lab 3

In [1]:
from datetime import datetime
registerPath = "/data/students/bigdata_internet/lab3/register.csv"
statiosPath = "/data/students/bigdata_internet/lab3/stations.csv"
outputPath = "s308885_Lab3_Solution/"

In [2]:
#reading the files from the paths
registerRdd = sc.textFile(registerPath)
stationRdd = sc.textFile(statiosPath)

In [3]:
#dividing data with \t
#removing the first row which is the header and we don't need it
newRegRdd = registerRdd.map(lambda x: x.split('\t'))
Regـheader = newRegRdd.first()
noHeaderRegRdd = newRegRdd.filter(lambda x: x!= Regـheader)

In [4]:
#filtering the data. since there are wrong data that free_slots and used_slots both are 0
# counting the number of data after cleaning
filteredRegRdd = noHeaderRegRdd.filter(lambda x: x[2]!="0" or x[3]!="0")
noHeaderRegRdd.count()

25319028

## 1.1.1
- number of remaining rows after filtering was 25319028 withought header
- number of remaining rows after filtering is 25104121 withought header

In [5]:
#splitting station data
#selecting the header
#removing the header
newStatRdd = stationRdd.map(lambda x: x.split('\t'))
Stat_header = newStatRdd.first()
noHeaderStatRdd = newStatRdd.filter(lambda x: x!= Stat_header)

# 2

- changing the style from existing date to (id, (day, hour), used solts, free slots)

In [6]:
#changing the timestamp to weekday and hour
# the data structure now is (station, weekday, hour, used_slots, free_slots)
slottedRdd = filteredRegRdd.map(lambda x: (x[0],(datetime.strftime(datetime.strptime(x[1], "%Y-%m-%d %H:%M:%S"),"%A %H").split(" ")[0], datetime.strftime(datetime.strptime(x[1], "%Y-%m-%d %H:%M:%S"),"%A %H").split(" ")[1]),x[2],x[3]))

In [7]:
#caching the data to be able to use it faster later
slottedRdd.cache()

PythonRDD[7] at RDD at PythonRDD.scala:53

In [8]:
#counting the data after cleaning
#result is 25104121
slottedRdd.count()

25104121

#### now we get the rdd grouped by the hours and the result would have 168 = 7*24 row

In [9]:
#grouping the data according to station_id and weekday to have all the rows related to same id , weekday and hour in the same group
#the structure is           [(('234', ('Friday', '17')),<pyspark.resultiterable.ResultIterable at 0x7f5967dcbd50>)]
grouped_rdd = slottedRdd.groupBy(lambda x: (x[0],x[1]))

In [11]:
#the second part after grouping is not readable so we change it to a list
listed_result_rdd = grouped_rdd.map(lambda x: (x[0], list(x[1])))

In [12]:
# a function to conut the number of zeros in each group to calculate the criticality
def iterrr(items):
    zeros = 0;
    for item in items:
        if item[3] == '0':
            zeros+=1
    return zeros

In [15]:
# now we add a column which is length of the array in the second index of each group
# to have number of the rows in each group
new_res = listed_result_rdd.map(lambda x: (x[0],x[1],len(x[1])))

In [16]:
# now we call the function to get the zeros of each group
result_with_zeros = new_res.map(lambda x: ( x[0], x[1], x[2], iterrr(x[1]) ))

In [17]:
# now that we have total rows and number of zeros in each group we can calculate the criticality
critical_ratio_rdd = result_with_zeros.map(lambda x: ( x[0], x[1], x[2], x[3], x[3]/x[2]))

In [18]:
##import sys
# min_threshold = sys.argv[0]
# if we want to pass the threshold we can use the argument which can be passed through terminal commands
min_threshold = 0.6

In [19]:
# we have a minimum threshold which is 0.6. now we filter the data (each group which has criticality rate over min threshold)
filtered_critical_rdd = critical_ratio_rdd.filter(lambda x: x[4]>min_threshold)

In [20]:
# next we want to sort by somw of the columns but problem is weekdays can't be sorted properly since they are strings
# so we make a key-value pair which we can use
weekdays = { 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5,'Saturday': 6, 'Sunday': 7}

In [21]:
# data is sorted by these items and priority is as follows
# first criticality_rate, station_id, weekday, hour
#for weekday we use the object we built in the previous step weekdays.get("key") => value which is a number and can be sorted
sorted_rdd = filtered_critical_rdd.sortBy(lambda x: (x[4], x[0][0], weekdays.get(x[0][1][0]),x[0][1][1]))

In [24]:
#now we have all the data we needed now we change the structure for further joins
#we change it to (key, value) whcich looks like this ('9', ('Friday', '10', 0.6129032258064516))
#then we can join two tables by their ids
noLattitude_rdd = sorted_rdd.map(lambda x: (x[0][0],(x[1][0][1][0],x[1][0][1][1], x[4])))

In [26]:
#now we need to change the structure of the station rdd to (id,(lat, long)) to make the join possible
station_rdd_pair = noHeaderStatRdd.map(lambda x : (x[0],(x[1],x[2])))

In [28]:
#now we joing two tables which is joined based on the key
#each row looks like this ('10', (('Saturday', '00', 0.622107969151671), ('2.185206', '41.384875')))
joined_rdd = noLattitude_rdd.join(station_rdd_pair)

In [30]:
#now we change the structure the way we need. so we take out the elements of each row according to example
#and each row looks like this ('10', '2.185206', '41.384875', 'Saturday', '00', 0.622107969151671)
final_joined_rdd = joined_rdd.map(lambda x: (x[0],x[1][1][0],x[1][1][1],x[1][0][0],x[1][0][1], x[1][0][2]))

In [31]:
final_joined_rdd.take(50)

[('10', '2.185206', '41.384875', 'Saturday', '00', 0.622107969151671),
 ('9', '2.185294', '41.385006', 'Friday', '22', 0.6258389261744967),
 ('9', '2.185294', '41.385006', 'Friday', '10', 0.6129032258064516),
 ('58', '2.170736', '41.377536', 'Monday', '00', 0.6323119777158774),
 ('58', '2.170736', '41.377536', 'Monday', '01', 0.6239554317548747)]

In [48]:
# to add the header we create an rdd with the column names
#obviously there are different way to do that
header = ("Station", "Lattitude", "Longtitude", "Weekday", "Hour", "Criticality")
header_rdd = sc.parallelize([header])

In [49]:
#we make a union to mix two rdds
result_rdd = header_rdd.union(final_joined_rdd)

In [50]:
#cast data to string then add \t as seperator
saveable_rdd = result_rdd.map(lambda x: str(x[0])+'\t'+str(x[1])+'\t'+str(x[2])+'\t'+str(x[3])+'\t'+str(x[4])+'\t'+str(x[5]))

In [47]:
#saving  the final result as csv file
saveable_rdd.coalesce(1).saveAsTextFile("lab3_s308885_final_result.csv/")

# the final result is this
[

     ('10', '2.185206', '41.384875', 'Saturday', '00', 0.622107969151671),

     ('9', '2.185294', '41.385006', 'Friday', '22', 0.6258389261744967),
     
     ('9', '2.185294', '41.385006', 'Friday', '10', 0.6129032258064516),
 
     ('58', '2.170736', '41.377536', 'Monday', '00', 0.6323119777158774),
 
     ('58', '2.170736', '41.377536', 'Monday', '01', 0.6239554317548747)
 
 ]

#-----------------------------------------------------------------------------------------------------------

### USING Spark SQL
### -------------------------------------------------------

In [85]:
#defining the paths
from pyspark.sql.functions import *
registerPath = "/data/students/bigdata_internet/lab3/register.csv"
statiosPath = "/data/students/bigdata_internet/lab3/stations.csv"
outputPath = "s308885_Lab3_Solution/"

In [86]:
# defining how to read the date with delimeter
registerDF = spark.read.csv(registerPath, header=True, inferSchema=True, sep= '\t')
stationDF = spark.read.csv(statiosPath, header=True, inferSchema=True, sep= '\t')

In [87]:
# - `hour(timestamp)` to obtain the hour
# - `date_format(timestamp,'EEEE')` to obtain the day of the week
# chaging the format of timestamp into two columns of weekday and hour
#removing the timestamp column. we don't need it
from datetime import datetime
registerDF = registerDF.withColumn("weekday", date_format(registerDF.timestamp, 'EEEE'))
registerDF = registerDF.withColumn("hour", hour(registerDF.timestamp))
registerDF = registerDF.drop("timestamp")

In [88]:
# filtering the data to remove row with the wrong data which is descried in the document
filtered_df = registerDF.filter("used_slots != 0 or free_slots != 0")

In [89]:
# defining two table name for two dataframes to use in SQL queries
filtered_df.createOrReplaceTempView("register")
stationDF.createOrReplaceTempView("station")

In [90]:
#total of row ramaining after filtering is 25104121
filtered_df.count()

25104121

In [91]:
# the schema related to data frame is as given
filtered_df.printSchema()

root
 |-- station: integer (nullable = true)
 |-- used_slots: integer (nullable = true)
 |-- free_slots: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)



In [92]:
# importing the function from pyspark sql module
# creating a column for zero values in free slots. if free_slots is zero then the value is 1 otherwise will be 0.
# we do this to count zeros after grouping the data
# we name the new table zeroRegister

from pyspark.sql import functions as F
withZeros_df = filtered_df.withColumn("zeros", F.when(filtered_df["free_slots"] == 0, 1).otherwise(0))
withZeros_df.createOrReplaceTempView("zeroRegister")

In [93]:
# now table is group by "station", "weekday", "hour"
# two columns are added. first one is total number of zeros
# second one is total number of rows for each group
# columns are called sum_zeros and count_free_slots
# finally we keep the criticality rate caculated using zeros and total which is named criticality

from pyspark.sql import functions as F
grouped_criticality = withZeros_df.groupBy(["station", "weekday", "hour"]) \
       .agg(F.sum("zeros").alias("sum_zeros"), F.count("free_slots").alias("count_free_slots")) \
       .selectExpr("station", "weekday", "hour", "sum_zeros / count_free_slots as criticality")
grouped_criticality.show()

[Stage 153:======================================>                  (4 + 2) / 6]

+-------+---------+----+--------------------+
|station|  weekday|hour|         criticality|
+-------+---------+----+--------------------+
|    193|Wednesday|  16|                 0.0|
|    194|   Friday|   6|                 0.0|
|    197|Wednesday|  23|                 0.0|
|    197| Saturday|   0|                 0.0|
|    201|  Tuesday|  14|                 0.0|
|    201|Wednesday|  10|0.001964636542239...|
|    202|   Sunday|  15|                 0.0|
|    202|   Monday|  13|                 0.0|
|    203|   Friday|   6| 0.01011804384485666|
|    204|  Tuesday|  19| 0.05016722408026756|
|    205| Saturday|  19|                 0.0|
|    205|   Sunday|   9|                 0.0|
|    205|   Monday|   7| 0.03302752293577982|
|    206|   Sunday|  13|0.012302284710017574|
|    208|  Tuesday|  15|                 0.1|
|    211|   Monday|   7|                 0.0|
|    212|   Friday|  23|                 0.0|
|    213|   Friday|  18|                0.05|
|    213| Thursday|   1| 0.0795755

#table looks like this

+-------+---------+----+-------------------+

|station|  weekday|hour|        criticality|

+-------+---------+----+-------------------+

|    263|Wednesday|  23|0.16111111111111112|

In [94]:
#minimum criticality is 0.6
MinimumCriticality = 0.6

In [95]:
# now we filter the groups which have criticality over the min threshold
threshold_df = grouped_criticality.select("*").where(grouped_criticality["criticality"] >= 0.6)

In [96]:
# two tables are joined with a condition threshold_df.station == stationDF.id
# then we select the rows we need. join bring all the columns which we don't need
joined_dfs = threshold_df.join(stationDF, threshold_df.station == stationDF.id).select(threshold_df.station, stationDF.longitude, stationDF.latitude, threshold_df.weekday, threshold_df.hour, threshold_df.criticality)

#now joined tables look like this
+-------+---------+---------+-------+----+------------------+
|station|longitude| latitude|weekday|hour|       criticality|
+-------+---------+---------+-------+----+------------------+
|     58| 2.170736|41.377536| Sunday|  23|0.6239554317548747|
|     10| 2.185206|41.384875| Friday|  22| 0.622107969151671|
|      9| 2.185294|41.385006| Friday|   8|0.6129032258064516|
|     58| 2.170736|41.377536| Sunday|  22|0.6323119777158774|
|      9| 2.185294|41.385006| Friday|  20|0.6258389261744967|
+-------+---------+---------+-------+----+------------------+

In [99]:
#importing the modules we need
# user-defined-functions
# integer type to define the new column schema type
# importing everything to be sure :)

# an object is defined to be used for sorting weekdays and the we create a UDF for that named weekday_order using weekdays.get(x)
# weekday_order is a function which returns an specific integer for each day

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql import *

weekdays = { 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5,'Saturday': 6, 'Sunday': 7}
# spark.udf.register("weekday_order", lambda x: weekdays.get(x), IntegerType()) //once worked then didn't
weekday_order = udf(lambda x: weekdays.get(x), IntegerType())

In [100]:
# create a column named weekday_id and the data is generated using weekday_order UDF

joined_dfs = joined_dfs.withColumn("weekday_id", weekday_order(joined_dfs.weekday))

In [101]:
# now we have all the data needed we can sort by "criticality","station","weekday_id","hour" which are all ascending so the second part is all True
sorted_df = joined_dfs.sort("criticality","station","weekday_id","hour",ascending=[True,True,True,True])

In [105]:
# weekday_id column is not needed so we drop it
final_DF = sorted_df.drop("weekday_id")

#final table looks like this:
+-------+---------+---------+-------+----+------------------+
|station|longitude| latitude|weekday|hour|       criticality|
+-------+---------+---------+-------+----+------------------+
|      9| 2.185294|41.385006| Friday|   8|0.6129032258064516|
|     10| 2.185206|41.384875| Friday|  22| 0.622107969151671|
|     58| 2.170736|41.377536| Sunday|  23|0.6239554317548747|
|      9| 2.185294|41.385006| Friday|  20|0.6258389261744967|
|     58| 2.170736|41.377536| Sunday|  22|0.6323119777158774|
+-------+---------+---------+-------+----+------------------+

In [109]:
# to add the header we create an rdd with the column names
#obviously there are different way to do that
header = ("Station", "Lattitude", "Longtitude", "Weekday", "Hour", "Criticality")
header_rdd = sc.parallelize([header])

#first we change df to rdd
df_to_rdd = final_DF.rdd

#we make a union to mix two rdds
result_rdd = header_rdd.union(df_to_rdd)


#cast data to string then add \t as seperator
saveable_rdd = result_rdd.map(lambda x: str(x[0])+'\t'+str(x[1])+'\t'+str(x[2])+'\t'+str(x[3])+'\t'+str(x[4])+'\t'+str(x[5]))

In [110]:
#saving  the final result as csv file
saveable_rdd.coalesce(1).saveAsTextFile("lab3_s308885_final_result.csv/")

['Station\tLattitude\tLongtitude\tWeekday\tHour\tCriticality',
 '9\t2.185294\t41.385006\tFriday\t8\t0.6129032258064516',
 '10\t2.185206\t41.384875\tFriday\t22\t0.622107969151671',
 '58\t2.170736\t41.377536\tSunday\t23\t0.6239554317548747',
 '9\t2.185294\t41.385006\tFriday\t20\t0.6258389261744967',
 '58\t2.170736\t41.377536\tSunday\t22\t0.6323119777158774']

In [111]:
# or we can use this method
#final_DF.write.csv("final_df_stations_seperated/", header=True,sep= '\t')